In [ ]:
import sqlite3
import pandas as pd

In [ ]:
con = sqlite3.connect("/content/telecom.db")
con

In [ ]:
cur = con.cursor()
cur

## 1 Retrieve table names from the database (telecom.db).

In [ ]:
res = cur.execute("SELECT name FROM sqlite_master WHERE type='table'")
res.fetchall()

[('train',),
 ('event_type',),
 ('severity_type',),
 ('resource_type',),
 ('log_feature',)]

In [ ]:
pd.read_sql("SELECT * FROM train", con)

,index,id,location,fault_severity
0,0,14121,location 118,1
1,1,9320,location 91,0
2,2,14394,location 152,1
3,3,8218,location 931,1
4,4,14804,location 120,0
...,...,...,...,...
7376,7376,870,location 167,0
7377,7377,18068,location 106,0
7378,7378,14111,location 1086,2
7379,7379,15189,location 7,0


## 2 Join all tables in the database on the primary key.

In [ ]:
data = pd.read_sql("""select  lf.id, lf.log_feature, lf.volume, et.event_type, rt.resource_type, st.severity_type, t.location, t.fault_severity
                  from log_feature as lf
                  left join event_type as et
                    on et.`index` = lf.`index`
                  left join resource_type as rt
                    on et.`index` = rt.`index`
                  left join severity_type as st
                    on et.`index` = st.`index`
                  left join train as t
                    on et.id = t.id""", con)
data

,id,log_feature,volume,event_type,resource_type,severity_type,location,fault_severity
0,6597,feature 68,6,event_type 11,resource_type 8,severity_type 2,None,NaN
1,8011,feature 68,7,event_type 15,resource_type 8,severity_type 2,location 1,0.0
2,2597,feature 68,1,event_type 15,resource_type 8,severity_type 2,None,NaN
3,5022,feature 172,2,event_type 15,resource_type 8,severity_type 1,None,NaN
4,5022,feature 56,1,event_type 11,resource_type 8,severity_type 1,None,NaN
...,...,...,...,...,...,...,...,...
58666,8720,feature 209,1,None,None,None,None,NaN
58667,6488,feature 54,3,None,None,None,None,NaN
58668,878,feature 62,1,None,None,None,None,NaN
58669,4464,feature 209,1,None,None,None,None,NaN


## 3 Find unique occurrences of event_type and severity using an SQL query.

In [ ]:
data = pd.read_sql("""select  distinct event_type, severity_type
                  from log_feature as lf
                  left join event_type as et
                    on et.`index` = lf.`index`
                  left join resource_type as rt
                    on et.`index` = rt.`index`
                  left join severity_type as st
                    on et.`index` = st.`index`
                  left join train as t
                    on et.id = t.id""", con)
data

,event_type,severity_type
0,event_type 11,severity_type 2
1,event_type 15,severity_type 2
2,event_type 15,severity_type 1
3,event_type 11,severity_type 1
4,event_type 20,severity_type 2
...,...,...
147,event_type 43,None
148,event_type 52,None
149,event_type 51,None
150,event_type 48,None


## 4 Find how many occurrences there are of each fault_severity using an SQL query.

In [ ]:
data = pd.read_sql("""select  fault_severity, count(*)
                  from train
                    group by fault_severity""", con)
data

,fault_severity,count(*)
0,0,4784
1,1,1871
2,2,726
